### Arbitraje de precios con técnicas de aprendizaje de máquina - Construcción Base de Datos

En este notebook desarrollamos el paso a paso de la construcción de las bases de datos que utilizaremos para la elaboración de los modelos de aprendizaje de máquina y aprendizaje profundo para técnicas de arbitraje de precio en el mercado de renta variable colombiano y los instrumentos con dual listing en mercados extranjeros. Nos enfocaremos específicamente en construir una base de datos que refleje el minuto a minuto de las condiciones de precio (último operado y mejores precios en el libro de ofertas) para la acción preferencial de Bancolombia que se negocia en pesos colombianos en la Bolsa de Valores de Colombia y su respectivo ADR (American depositary receipt) que se negocia en dólares americanos en la bolsa de NY. Dado que las condiciones de arbitraje se presentan cuando se comparan los precios para el instrumento y su instrumento dual, se requiere un parámetro de compraración. Es por esta razón que incluiremos en la base de datos el comportamiento histórico de los precios (último operado y mejores precios en el libro de ofertas), minuto a minuto, para la tasa de cambio del peso colombiano respecto del dólar americano negociado en sistema de referencia SetFx, el cual es el principal sistema de negociación de la divisa para el mercado de capitales colombiano.

In [1]:
#Imports
import pandas as pd
import numpy as np
import eikon as ek
import refinitiv.data as rd
import datetime
from IPython.display import display, clear_output
from refinitiv.data.content import historical_pricing
from refinitiv.data.content.historical_pricing import Intervals
from refinitiv.data.content.historical_pricing import Adjustments
from refinitiv.data.content.historical_pricing import MarketSession
import datetime

In [2]:
#App Key Eikon
ek.set_app_key('5b95039450074aedaf4e1dc8418ba45ecca6b27d')

In [3]:
#Imports
import pytz
import time
from datetime import datetime, timezone
 
currenttime_str = time.ctime()
print(currenttime_str)

Sun Oct 23 21:32:29 2022


In [4]:
#Definimos las zonas horarias con las cuales trabajemos 
bogota = pytz.timezone('America/Bogota')
ny=pytz.timezone('America/New_York')

### Datos

Extraemos los datos de precio de las APIs Eikon (Refinitiv), utilizando el appkey proporcionado por el proveedor de información.

In [5]:
#Cib dayly, minute interval
cib = ek.get_timeseries(["CIB"], fields='CLOSE', start_date="2022-01-03", end_date="2022-05-14", interval='minute', 
count=None, calendar='calendardays', corax=None, normalize=False, raw_output=False, 
debug=False)
cib

CIB,CLOSE
Date,
2022-01-03 14:31:00,31.63
2022-01-03 14:33:00,31.3701
2022-01-03 14:36:00,31.6839
2022-01-03 14:38:00,31.72
2022-01-03 14:39:00,31.745
...,...
2022-05-13 20:00:00,37.49
2022-05-13 20:01:00,37.5
2022-05-13 20:11:00,37.5


In [6]:
#Timestamp según horario en la BVC
CIB_close=cib.tz_localize(pytz.utc).tz_convert(bogota)

In [7]:
CIB_close.head()

CIB,CLOSE
Date,
2022-01-03 09:31:00-05:00,31.63
2022-01-03 09:33:00-05:00,31.3701
2022-01-03 09:36:00-05:00,31.6839
2022-01-03 09:38:00-05:00,31.72
2022-01-03 09:39:00-05:00,31.745


In [8]:
#cambiamos el nombre de la columna de precios
CIB_close.columns=['CIB_close']

In [9]:
#USD/COP spot rate dayly
cop = ek.get_timeseries(["COP=STFX"], fields='CLOSE', start_date="2022-01-03", end_date="2022-05-14", interval='minute', 
count=None, calendar='calendardays', corax=None, normalize=False, raw_output=False, 
debug=False)
cop

COP=STFX,CLOSE
Date,
2022-01-03 13:22:00,4050.0
2022-01-03 13:36:00,4042.0
2022-01-03 13:40:00,4050.0
2022-01-03 13:42:00,4050.0
2022-01-03 13:44:00,4050.0
...,...
2022-05-13 17:55:00,4107.8
2022-05-13 17:56:00,4110.4
2022-05-13 17:57:00,4110.2


In [10]:
#Timestamp según horario BVC
COP_close=cop.tz_localize(pytz.utc).tz_convert(bogota)

In [11]:
COP_close.tail()

COP=STFX,CLOSE
Date,
2022-05-13 12:55:00-05:00,4107.8
2022-05-13 12:56:00-05:00,4110.4
2022-05-13 12:57:00-05:00,4110.2
2022-05-13 12:59:00-05:00,4110.0
2022-05-13 13:00:00-05:00,4110.5


In [12]:
#Cambiamos el nombre de la columna
COP_close.columns=['COP_close']

In [13]:
#PFBCOLOM dayly
pfb = ek.get_timeseries(["BIC_p1.CN"], fields='CLOSE', start_date="2022-01-03", end_date="2022-05-14", interval='minute', 
count=None, calendar=None, corax=None, normalize=False, raw_output=False, 
debug=False)
pfb

BIC_p1.CN,CLOSE
Date,
2022-01-03 14:54:00,32000
2022-01-03 14:55:00,32000
2022-01-03 14:56:00,32300
2022-01-03 15:01:00,32290
2022-01-03 15:02:00,32290
...,...
2022-05-13 19:47:00,38410
2022-05-13 19:51:00,38440
2022-05-13 19:52:00,38440


In [14]:
#Timestamp según horario BVC
PFB_close=pfb.tz_localize(pytz.utc).tz_convert(bogota)

In [15]:
PFB_close.columns=['PFB_close']

In [16]:
#Next day dayly
next_day = ek.get_timeseries(["COP1DBP=STFX"], fields='CLOSE', start_date="2022-01-03", end_date="2022-05-14", interval='minute', 
count=None, calendar=None, corax=None, normalize=False, raw_output=False, 
debug=False)
next_day

COP1DBP=STFX,CLOSE
Date,
2022-01-03 18:04:00,4053.0
2022-01-03 18:05:00,4060.0
2022-01-03 18:21:00,4062.0
2022-01-03 18:45:00,4060.0
2022-01-04 18:02:00,4079.0
...,...
2022-05-13 19:33:00,4108.0
2022-05-13 20:32:00,4107.0
2022-05-13 20:42:00,4105.0


In [17]:
#Timestamp según horario Bogotá
next_day_close=next_day.tz_localize(pytz.utc).tz_convert(bogota)

In [18]:
next_day_close.head()

COP1DBP=STFX,CLOSE
Date,
2022-01-03 13:04:00-05:00,4053.0
2022-01-03 13:05:00-05:00,4060.0
2022-01-03 13:21:00-05:00,4062.0
2022-01-03 13:45:00-05:00,4060.0
2022-01-04 13:02:00-05:00,4079.0


In [19]:
#Cambiamos el nombre de la columna
next_day_close.columns=['next_day_close']

Para nuestro ejecicio, consideraremos una variable exógena: el índice de referencia Dow Jones de industriales, e incluiremos en la base de datos, el precio de cierre minuto a minuto del valor del índice.

In [20]:
#SPX
SPX = ek.get_timeseries([".DJI"], fields='CLOSE', start_date="2022-01-03", end_date="2022-05-14", interval='minute', 
count=None, calendar=None, corax=None, normalize=False, raw_output=False, 
debug=False)
SPX

.DJI,CLOSE
Date,
2022-01-03 14:31:00,36413.76
2022-01-03 14:32:00,36417.99
2022-01-03 14:33:00,36398.62
2022-01-03 14:34:00,36426.1
2022-01-03 14:35:00,36455.79
...,...
2022-05-13 20:18:00,32196.66
2022-05-13 20:19:00,32196.66
2022-05-13 20:20:00,32196.66


In [21]:
#Timestamp según horario Bogotá
SPX_close=SPX.tz_localize(pytz.utc).tz_convert(bogota)

In [22]:
SPX_close.head()

.DJI,CLOSE
Date,
2022-01-03 09:31:00-05:00,36413.76
2022-01-03 09:32:00-05:00,36417.99
2022-01-03 09:33:00-05:00,36398.62
2022-01-03 09:34:00-05:00,36426.1
2022-01-03 09:35:00-05:00,36455.79


In [23]:
#Cambiamos el nombre de la columna de precios
SPX_close.columns=['SPX_close']

### Pre-procesamiento y construcción del Time Index para las series de tiempo 

Las condiciones de negociación en el tiempo para los mercados de Colombia y la bolsa NY no son las mismas. En primer lugar, algunos instumentos como la tasa de cambio USD/COP tienen diferentes horarios de negociación según el tipo de liquidación de las operaciones. En el caso del USD/COP, tenemos por ejemplo, el mercado spot cuya liquidación se realiza el mismo día de la negociación y, que va desde las 8a.m. hasta la 1:00 p.m. horario de Bogotá; y el horario de negociación con liquidación next-day, que termina a las 4:30 p.m. hora Bogotá. Las operaciones de cobertura cambiaria para las operaciones de arbitraje de precio suelen tener como referencia la negociaión USD/COP del mercado spot hasta la 1:00p.m. y como referencia la negociación next-day para las operaciones que van de la 1:00p.m. hasta las 4:30 p.m. Los días festivos en EE.UU. no hay negociación spot para la tasa de cambio USD/COP y todo se liquida en next-day, por ejemplo. 
Por su parte, la Bolsa de Valores de Colombia y la Bolsa de NY negocian en diferentes condiciones de tiempo y tipo de operación: por ejemplo, los días festivos en sus respectivos países los mercados de renta variable permanecen cerrados, pero en el dual listing puede que el mercado permanezca abierto. Así mismo, la Bolsa de Valores de Colombia auqnque negocia en sesión de calce continuo durante la mayor parte de la jornada, tiene un mecanismo de negociación por subasta que se activa durante los últimos cinco minutos de cada rueda de negociación. La bolsa de NY por su parte, negocia en calce continuo durante toda la jornada y tiene sesiones de pre y after market, antes y después de cada jornada, respectivamente. De otro lado, ambas bolsas operan simultáneamente, lo cual implica que la Bolsa de Valores de Colombia tenga que ajustar su horario de operación según si en EE.UU. hay horario de verano u horario de invierno.  
La base de datos debe reflejar estas condiciones, puesto que estas condiciones determinan la ocurrencia de los arbitrajes. Es por esta razón que agregamos a la base de datos, variables ordinales y/o booleanas que reflejan estas características particulares.

### Festivos

In [24]:
import holidays

In [25]:
#Festivos EE.UU en 2022
for holiday in holidays.US(years=[2022]).items():
    print(holiday)

(datetime.date(2022, 1, 1), "New Year's Day")
(datetime.date(2022, 1, 17), 'Martin Luther King Jr. Day')
(datetime.date(2022, 2, 21), "Washington's Birthday")
(datetime.date(2022, 5, 30), 'Memorial Day')
(datetime.date(2022, 6, 19), 'Juneteenth National Independence Day')
(datetime.date(2022, 6, 20), 'Juneteenth National Independence Day (Observed)')
(datetime.date(2022, 7, 4), 'Independence Day')
(datetime.date(2022, 9, 5), 'Labor Day')
(datetime.date(2022, 10, 10), 'Columbus Day')
(datetime.date(2022, 11, 11), 'Veterans Day')
(datetime.date(2022, 11, 24), 'Thanksgiving')
(datetime.date(2022, 12, 25), 'Christmas Day')
(datetime.date(2022, 12, 26), 'Christmas Day (Observed)')


In [26]:
#Festivos bursátiles NYSE (used by all other US-exchanges, including NASDAQ, etc)
for holiday in holidays.XNYS(years=[2022]).items():
    print(holiday)

(datetime.date(2022, 1, 3), "New Year's Day (Observed)")
(datetime.date(2022, 1, 17), 'Martin Luther King Jr. Day')
(datetime.date(2022, 2, 21), "Washington's Birthday")
(datetime.date(2022, 4, 15), 'Good Friday')
(datetime.date(2022, 5, 30), 'Memorial Day')
(datetime.date(2022, 6, 20), 'Juneteenth National Independence Day (Observed)')
(datetime.date(2022, 7, 4), 'Independence Day')
(datetime.date(2022, 9, 5), 'Labor Day')
(datetime.date(2022, 11, 24), 'Thanksgiving Day')
(datetime.date(2022, 12, 26), 'Christmas Day (Observed)')


In [27]:
#Festivos Colombia
for holiday in holidays.Colombia(years=[2022]).items():
    print(holiday)

(datetime.date(2022, 5, 1), 'Día del Trabajo [Labour Day]')
(datetime.date(2022, 7, 20), 'Día de la Independencia [Independence Day]')
(datetime.date(2022, 8, 7), 'Batalla de Boyacá [Battle of Boyacá]')
(datetime.date(2022, 12, 8), 'La Inmaculada Concepción [Immaculate Conception]')
(datetime.date(2022, 12, 25), 'Navidad [Christmas]')
(datetime.date(2022, 1, 10), 'Día de los Reyes Magos [Epiphany](Observed)')
(datetime.date(2022, 3, 21), "Día de San José [Saint Joseph's Day](Observed)")
(datetime.date(2022, 7, 4), 'San Pedro y San Pablo [Saint Peter and Saint Paul](Observed)')
(datetime.date(2022, 8, 15), 'La Asunción [Assumption of Mary]')
(datetime.date(2022, 10, 17), 'Día de la Raza [Columbus Day](Observed)')
(datetime.date(2022, 11, 7), "Día de Todos los Santos [All Saint's Day](Observed)")
(datetime.date(2022, 11, 14), 'Independencia de Cartagena [Independence of Cartagena](Observed)')
(datetime.date(2022, 4, 14), 'Jueves Santo [Maundy Thursday]')
(datetime.date(2022, 4, 15), 'Vie

In [28]:
#Festivos en formato lista
holidays_COL=list(holidays.Colombia(years=[2022]).keys())
holidays_EEUU=list(holidays.US(years=[2022]).keys())

In [29]:
#Festivos en formato pd.timestamp
Festivos_COL=[pd.Timestamp(holidays_COL[t]) for t in range(len(holidays_COL))]
Festivos_NYSE=[pd.Timestamp(holidays_EEUU[t]) for t in range(len(holidays_EEUU))]

Creamos un timestamp minuto a minuto, que utilizaremos como vector de índices para nuestras series temporales. El rango de fechas irá día por día desde el 2022/1/3 hasta el 2022/5/13. Cada día, consideramos la franja horaria desde 8:30 hasta las 17:00.

In [30]:
#Vector de timestamsp en formato numérico para la base de datos desde 2022/1/3 a las 8:30 a.m hasta 2022/5/13 a las 17:00
from datetime import datetime, timedelta

def datetime_range(start, end, delta):
    current = start
    while current < end:
        if current.hour>=8 and current.hour< 17 and current.weekday() < 5:
            yield current
        current += delta

dts = [dt.strftime('%Y-%m-%d T%H:%M')  for dt in 
       datetime_range(datetime(2022, 1, 3, 8,30), datetime(2022, 5, 13, 17,0), 
       timedelta(minutes=1))] #dt.timetamp() #strftime('%Y-%m-%d T%H:%M Z')


In [31]:
#dts vector de timestamps para el dataset (minuto a minuto)
dts

['2022-01-03 T08:30',
 '2022-01-03 T08:31',
 '2022-01-03 T08:32',
 '2022-01-03 T08:33',
 '2022-01-03 T08:34',
 '2022-01-03 T08:35',
 '2022-01-03 T08:36',
 '2022-01-03 T08:37',
 '2022-01-03 T08:38',
 '2022-01-03 T08:39',
 '2022-01-03 T08:40',
 '2022-01-03 T08:41',
 '2022-01-03 T08:42',
 '2022-01-03 T08:43',
 '2022-01-03 T08:44',
 '2022-01-03 T08:45',
 '2022-01-03 T08:46',
 '2022-01-03 T08:47',
 '2022-01-03 T08:48',
 '2022-01-03 T08:49',
 '2022-01-03 T08:50',
 '2022-01-03 T08:51',
 '2022-01-03 T08:52',
 '2022-01-03 T08:53',
 '2022-01-03 T08:54',
 '2022-01-03 T08:55',
 '2022-01-03 T08:56',
 '2022-01-03 T08:57',
 '2022-01-03 T08:58',
 '2022-01-03 T08:59',
 '2022-01-03 T09:00',
 '2022-01-03 T09:01',
 '2022-01-03 T09:02',
 '2022-01-03 T09:03',
 '2022-01-03 T09:04',
 '2022-01-03 T09:05',
 '2022-01-03 T09:06',
 '2022-01-03 T09:07',
 '2022-01-03 T09:08',
 '2022-01-03 T09:09',
 '2022-01-03 T09:10',
 '2022-01-03 T09:11',
 '2022-01-03 T09:12',
 '2022-01-03 T09:13',
 '2022-01-03 T09:14',
 '2022-01-

In [32]:
#Cantidad de registros
len(dts)

51270

In [33]:
#agregamos la zona horaria al array de timestamps
pd.Timestamp(dts[0],tz=bogota)

Timestamp('2022-01-03 08:30:00-0500', tz='America/Bogota')

In [34]:
#Creamos array de pandas timestamps con su respectiva zona horaria
timestamp_base=[pd.Timestamp(dts[t],tz=bogota) for t in range(len(dts))]
timestamp_ny=[pd.Timestamp(dts[t],tz=ny) for t in range(len(dts))]

In [35]:
timestamp_base[0:5]

[Timestamp('2022-01-03 08:30:00-0500', tz='America/Bogota'),
 Timestamp('2022-01-03 08:31:00-0500', tz='America/Bogota'),
 Timestamp('2022-01-03 08:32:00-0500', tz='America/Bogota'),
 Timestamp('2022-01-03 08:33:00-0500', tz='America/Bogota'),
 Timestamp('2022-01-03 08:34:00-0500', tz='America/Bogota')]

In [36]:
timestamp_ny[0:5]

[Timestamp('2022-01-03 08:30:00-0500', tz='America/New_York'),
 Timestamp('2022-01-03 08:31:00-0500', tz='America/New_York'),
 Timestamp('2022-01-03 08:32:00-0500', tz='America/New_York'),
 Timestamp('2022-01-03 08:33:00-0500', tz='America/New_York'),
 Timestamp('2022-01-03 08:34:00-0500', tz='America/New_York')]

In [37]:
len(timestamp_ny)

51270

In [38]:
#Filtramos los precios del USD/COP next day por su horario de negociación principar entre las 13:00 y las 16:30
nxt= next_day_close.loc[(next_day_close.index.hour >= 13) & (next_day_close.index.hour <= 17)]

In [39]:
nxt

,next_day_close
Date,
2022-01-03 13:04:00-05:00,4053.0
2022-01-03 13:05:00-05:00,4060.0
2022-01-03 13:21:00-05:00,4062.0
2022-01-03 13:45:00-05:00,4060.0
2022-01-04 13:02:00-05:00,4079.0
...,...
2022-05-13 14:33:00-05:00,4108.0
2022-05-13 15:32:00-05:00,4107.0
2022-05-13 15:42:00-05:00,4105.0


### DataFrame de tipos de Fechas, Horas de mercado, Tipo de Mercado

Creamos el dataframe 'Fechas' que contiene las variables correspondientes a las características particulares de negociación en los diferentes mercados con los sigueintes features para el índice de timestamps creado en el numeral anterior:   
Date: Fecha  
Hora_COL: Hora en Colombia.  
Minuto: Minuto correspondiente al índice de timestamp respectivo.  
Hora_NY: Hora en N.Y.    
Festivo_COL: Columna booleana en la que se maracarán con "1" los días festivos en Colombia.  
Festivo_NYSE: Columna booleana en la que marcarán con "1" los días festivos en EE.UU.  
Next_Day: Columna booleana en la que se marcarán con "1" los timestamps que pertenzcan al horario de negociación next-day de la tasa de cambio. (13:00 a 16:30)  
Subasta: Columna booleana en la que se marcarán con "1" los timestamps que pertenezcan a la subasta de cierre de la BVC. (últimos cinco minutos +/- 1 minuto).  
PFB_ABIERTO: Columna booleana en la que se marcarán con "1" los timestamps en los cuales se puede enviar una orden para negociar acciones preferenciales de Bancolombia en la BVC.  
CIB_ABIERTO: Columna booleana en la que se marcarán con "1" los timesstamps en los cuales se puede enviar una orden para negociar acciones CIB en la bolsa de NY.    

In [40]:
#Tomamos el timestamp base construido en el punto anterior y agregamos columnas de tipo de fecha y tipo de sesión
Fechas=pd.DataFrame(columns=['Date','Hora_COL','Minuto','Hora_NY','Festivo_COL','Festivo_NYSE','Next_Day','Subasta', 'PFB_ABIERTO','CIB_ABIERTO'],index=timestamp_base)

In [41]:
Fechas

,Date,Hora_COL,Minuto,Hora_NY,Festivo_COL,Festivo_NYSE,Next_Day,Subasta,PFB_ABIERTO,CIB_ABIERTO
2022-01-03 08:30:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-03 08:31:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-03 08:32:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-03 08:33:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-03 08:34:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-05-13 16:55:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-13 16:56:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-13 16:57:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-13 16:58:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
#Agregamos columnas de hora y minuto
Fechas['Hora_COL']=[Fechas.index[i].hour for i in range(len(Fechas))]
Fechas['Minuto']=[Fechas.index[i].minute for i in range(len(Fechas))]

In [43]:
Fechas.index[0]

Timestamp('2022-01-03 08:30:00-0500', tz='America/Bogota')

### Diferencias de Horario Bogotá-NY

In [44]:
#Creamos la columna Hora_NY (teniendo en cuenta que los horarios de Bogotá y NY no simpre son iguales)
for i in range(len(Fechas)):
    if Fechas.index[i].value!=timestamp_ny[i].value:
        Fechas['Hora_NY'].iloc[i]=Fechas['Hora_COL'].iloc[i]+1
    else:
        Fechas['Hora_NY'].iloc[i]=Fechas['Hora_COL'].iloc[i]

C:\Users\fract\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


### Horario Next Day para la negociación de divisas

In [45]:
#Creamos la columna de Next_day (horario después de la 1:00 p.m). Marcamos la serie de tiempo de forma booleana.
for i in range(len(Fechas)):
    if Fechas.index[i].hour >=13:
        Fechas['Next_Day'].iloc[i]=1
    else:
        Fechas['Next_Day'].iloc[i]=0
    

### Horario para la subasta de cierre del mercado de acciones colombiano

In [46]:
#Creamos la columna de Subasta (horario de 2:54p.m a 3:00 p.m. para horario de verano, 3:54p.m a 4:00 p.m. horario de invierno)
for i in range(len(Fechas)):
    if Fechas['Hora_COL'].iloc[i]!=Fechas['Hora_NY'].iloc[i]: 
        if (Fechas.index[i].hour==14 and Fechas.index[i].minute>=54) or (Fechas.index[i].hour==15 and Fechas.index[i].minute==0):
            Fechas['Subasta'].iloc[i]=1
        else:
            Fechas['Subasta'].iloc[i]=0
    else:
        if (Fechas.index[i].hour==15 and Fechas.index[i].minute>=54) or (Fechas.index[i].hour==16 and Fechas.index[i].minute==0):
            Fechas['Subasta'].iloc[i]=1
        else:
            Fechas['Subasta'].iloc[i]=0

### Preprocesamiento para días festivos Bogotá-NY

In [47]:
#Creamos vectores de datetimes para los festivos para compararlos posteriormente y así alimentar el dataset a partir de ellos
col_fest=[]
nyse_fest=[]
timestamp_date=[]
for i in range(len(Festivos_COL)):
    col_fest.append(Festivos_COL[i].date())
for j in range(len(Fechas.index)):
    timestamp_date.append(Fechas.index[j].date())
for k in range(len(Festivos_NYSE)):
    nyse_fest.append(Festivos_NYSE[k].date())    

In [48]:
Fechas['Date']=timestamp_date

In [49]:
#Creamos fechas de festivos en Colombia y Festivos en EE.UU, las marcamos de forma boleana
Fechas['Festivo_COL'] = Fechas['Date'].isin(col_fest)
Fechas['Festivo_NYSE']=Fechas['Date'].isin(nyse_fest)

In [50]:
#Observamos
Fechas.tail(5)

,Date,Hora_COL,Minuto,Hora_NY,Festivo_COL,Festivo_NYSE,Next_Day,Subasta,PFB_ABIERTO,CIB_ABIERTO
2022-05-13 16:55:00-05:00,2022-05-13,16,55,17,False,False,1,0,NaN,NaN
2022-05-13 16:56:00-05:00,2022-05-13,16,56,17,False,False,1,0,NaN,NaN
2022-05-13 16:57:00-05:00,2022-05-13,16,57,17,False,False,1,0,NaN,NaN
2022-05-13 16:58:00-05:00,2022-05-13,16,58,17,False,False,1,0,NaN,NaN
2022-05-13 16:59:00-05:00,2022-05-13,16,59,17,False,False,1,0,NaN,NaN


### Sesiones de mercado abierto Bogotá-NY

In [51]:
#Creamos una columna para marcar la sesión abierta de la rueda de negociación de acciones Colombia
for i in range(len(Fechas)):
   
    if Fechas['Hora_COL'].iloc[i]!=Fechas['Hora_NY'].iloc[i]:
        if Fechas['Festivo_COL'].iloc[i]==True:
            Fechas['PFB_ABIERTO'].iloc[i]=0    
        elif (Fechas['Hora_COL'].iloc[i]<8) or (Fechas['Hora_COL'].iloc[i]==8 and Fechas['Minuto'].iloc[i]<30) or (Fechas['Hora_COL'].iloc[i]>=15):
            Fechas['PFB_ABIERTO'].iloc[i]=0
        else:
            Fechas['PFB_ABIERTO'].iloc[i]=1
    if Fechas['Hora_COL'].iloc[i]==Fechas['Hora_NY'].iloc[i]:
        if Fechas['Festivo_COL'].iloc[i]==True:
            Fechas['PFB_ABIERTO'].iloc[i]=0        
        elif (Fechas['Hora_COL'].iloc[i]<9) or (Fechas['Hora_COL'].iloc[i]==9 and Fechas['Minuto'].iloc[i]<30) or (Fechas['Hora_COL'].iloc[i]>=16):
            Fechas['PFB_ABIERTO'].iloc[i]=0
        else:
            Fechas['PFB_ABIERTO'].iloc[i]=1

C:\Users\fract\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [52]:
#creamos una columna para marcar la sesión abierta en la bolsa de NY según horario colombiano.
for i in range(len(Fechas)):
    if Fechas['Hora_COL'].iloc[i]!=Fechas['Hora_NY'].iloc[i]:
        if Fechas['Festivo_NYSE'].iloc[i]==True:
            Fechas['CIB_ABIERTO'].iloc[i]=0
        elif (Fechas['Hora_COL'].iloc[i]<8) or (Fechas['Hora_COL'].iloc[i]==8 and Fechas['Minuto'].iloc[i]<30) or (Fechas['Hora_COL'].iloc[i]>=15):
            Fechas['CIB_ABIERTO'].iloc[i]=0
        else:
            Fechas['CIB_ABIERTO'].iloc[i]=1
    if Fechas['Hora_COL'].iloc[i]==Fechas['Hora_NY'].iloc[i]:
        if Fechas['Festivo_NYSE'].iloc[i]==True:
            Fechas['CIB_ABIERTO'].iloc[i]=0
        elif (Fechas['Hora_COL'].iloc[i]<9) or (Fechas['Hora_COL'].iloc[i]==9 and Fechas['Minuto'].iloc[i]<30) or (Fechas['Hora_COL'].iloc[i]>=16):
            Fechas['CIB_ABIERTO'].iloc[i]=0
        else:
            Fechas['CIB_ABIERTO'].iloc[i]=1

In [53]:
Fechas.head(5)

,Date,Hora_COL,Minuto,Hora_NY,Festivo_COL,Festivo_NYSE,Next_Day,Subasta,PFB_ABIERTO,CIB_ABIERTO
2022-01-03 08:30:00-05:00,2022-01-03,8,30,8,False,False,0,0,0,0
2022-01-03 08:31:00-05:00,2022-01-03,8,31,8,False,False,0,0,0,0
2022-01-03 08:32:00-05:00,2022-01-03,8,32,8,False,False,0,0,0,0
2022-01-03 08:33:00-05:00,2022-01-03,8,33,8,False,False,0,0,0,0
2022-01-03 08:34:00-05:00,2022-01-03,8,34,8,False,False,0,0,0,0


In [54]:
Fechas.tail(5)

,Date,Hora_COL,Minuto,Hora_NY,Festivo_COL,Festivo_NYSE,Next_Day,Subasta,PFB_ABIERTO,CIB_ABIERTO
2022-05-13 16:55:00-05:00,2022-05-13,16,55,17,False,False,1,0,0,0
2022-05-13 16:56:00-05:00,2022-05-13,16,56,17,False,False,1,0,0,0
2022-05-13 16:57:00-05:00,2022-05-13,16,57,17,False,False,1,0,0,0
2022-05-13 16:58:00-05:00,2022-05-13,16,58,17,False,False,1,0,0,0
2022-05-13 16:59:00-05:00,2022-05-13,16,59,17,False,False,1,0,0,0


In [55]:
#Para exportar a excel (Excel no admite datos tipo pd.timestamp)
Fechas1=Fechas.reset_index()

In [56]:
Fechas1=Fechas1.drop(['index'],axis=1)

In [57]:
Fechas1.to_excel("Fechas.xlsx")

### Concatenar condiciones de horario con variables numéricas: close prices

In [58]:
#Creamos una base de datos, concatenando los precios de los activos y las caracterísitcas de horario
base_cib=pd.concat([Fechas,CIB_close,COP_close,PFB_close,next_day_close,SPX_close],axis=1,join='outer')

In [59]:
base_cib.head(5)

,Date,Hora_COL,Minuto,Hora_NY,Festivo_COL,Festivo_NYSE,Next_Day,Subasta,PFB_ABIERTO,CIB_ABIERTO,CIB_close,COP_close,PFB_close,next_day_close,SPX_close
2022-01-03 08:22:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,4050.0,<NA>,<NA>,<NA>
2022-01-03 08:30:00-05:00,2022-01-03,8.0,30.0,8,False,False,0,0,0,0,<NA>,<NA>,<NA>,<NA>,<NA>
2022-01-03 08:31:00-05:00,2022-01-03,8.0,31.0,8,False,False,0,0,0,0,<NA>,<NA>,<NA>,<NA>,<NA>
2022-01-03 08:32:00-05:00,2022-01-03,8.0,32.0,8,False,False,0,0,0,0,<NA>,<NA>,<NA>,<NA>,<NA>
2022-01-03 08:33:00-05:00,2022-01-03,8.0,33.0,8,False,False,0,0,0,0,<NA>,<NA>,<NA>,<NA>,<NA>


In [60]:
#creamos una columna 'tasa de cambio' para unificar los precios spot y next day de la tasa USD/COP
base_cib['tasa_cambio']=[base_cib['COP_close'].iloc[i] if base_cib['Next_Day'].iloc[i]==0 else base_cib['next_day_close'].iloc[i] for i in range(len(base_cib))] 

In [61]:
#Contamos los datos en los que no hay cambio de precio
base_cib['tasa_cambio'].isnull().sum()

27609

In [62]:
#Tamaño de la base de precios
base_cib.shape

(51531, 16)

In [63]:
#Columnas
base_cib.columns

Index(['Date', 'Hora_COL', 'Minuto', 'Hora_NY', 'Festivo_COL', 'Festivo_NYSE',
       'Next_Day', 'Subasta', 'PFB_ABIERTO', 'CIB_ABIERTO', 'CIB_close',
       'COP_close', 'PFB_close', 'next_day_close', 'SPX_close', 'tasa_cambio'],
      dtype='object')

### Precios BID - ASK Para las variables endógenas

A continuación, agregamos los mejores precios disponibles en el libro de ofertas para la serie de tiempo correspondiente a la base de datos construida en el numeral anterior.  
Para obtener los precios históricos de los instrumentos de renta variable utilizaremos el API Refiniv data y para los precios del libro de ofertas de la tasa de cambio, una base de datos con el histórico de variaciones de este libro, obtenido directamente de SetFx.

In [64]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x1f2b5ecbc10 {name='workspace'}>

In [65]:
#Mejores precios para el libro de ofertas en el periodo 2022/1/3 - 2022/5/13
CIB_mid=rd.get_history("CIB",fields=["BID", "ASK"], interval="minute",start="2022-01-03",end="2022-05-14")
PFB_mid=rd.get_history("BIC_p1.CN",fields=["BID", "ASK"], interval="minute",start="2022-01-03",end="2022-05-14")

In [66]:
CIB_mid

CIB,ASK,BID
Timestamp,,
2022-01-03 09:00:00,40.0,18.88
2022-01-03 12:00:00,36.5,18.88
2022-01-03 14:20:00,33.03,31.0
2022-01-03 14:26:00,31.98,31.28
2022-01-03 14:27:00,31.98,31.08
...,...,...
2022-05-13 19:57:00,37.46,37.43
2022-05-13 19:58:00,37.48,37.46
2022-05-13 19:59:00,37.5,37.49


In [67]:
PFB_mid

BIC_p1.CN,ASK,BID
Timestamp,,
2022-01-03 14:15:00,32300,31700
2022-01-03 14:30:00,32300,31800
2022-01-03 14:31:00,32300,31820
2022-01-03 14:32:00,32300,31860
2022-01-03 14:33:00,32300,31910
...,...,...
2022-05-13 19:54:00,38480,38440
2022-05-13 19:55:00,35090,39950
2022-05-13 19:56:00,35090,39950


In [68]:
#Timestamp según horario Bogotá
CIB_mid=CIB_mid.tz_localize(pytz.utc).tz_convert(bogota)
PFB_mid=PFB_mid.tz_localize(pytz.utc).tz_convert(bogota)

In [69]:
CIB_mid

CIB,ASK,BID
Timestamp,,
2022-01-03 04:00:00-05:00,40.0,18.88
2022-01-03 07:00:00-05:00,36.5,18.88
2022-01-03 09:20:00-05:00,33.03,31.0
2022-01-03 09:26:00-05:00,31.98,31.28
2022-01-03 09:27:00-05:00,31.98,31.08
...,...,...
2022-05-13 14:57:00-05:00,37.46,37.43
2022-05-13 14:58:00-05:00,37.48,37.46
2022-05-13 14:59:00-05:00,37.5,37.49


In [70]:
#Cambiamos el nombre de las columnas
CIB_mid.columns=['ASK_CIB','BID_CIB']
PFB_mid.columns=['ASK_PFB','BID_PFB']

### Precios BID-ASK para la tasa de cambio USD/COP

In [71]:
#Abrimos el histórico de cambios en libro de mejores ofertas para la tasa de cambio
USD_feather=pd.read_feather('usd.feather')

In [72]:
USD_feather.head()

,Hora,Demanda,Oferta,Fecha
0,08:00:24,NaN,4100.0,2022-01-03 08:00:24
1,08:00:24,3800.0,4100.0,2022-01-03 08:00:24
2,08:00:31,3850.0,4100.0,2022-01-03 08:00:31
3,08:00:35,3900.0,4100.0,2022-01-03 08:00:35
4,08:00:44,3901.0,4100.0,2022-01-03 08:00:44


In [73]:
#Indexamos el USD.feather según los timestamps de la base que construimos en el numeral anterior
USD_feather['Fecha']=[pd.Timestamp(USD_feather['Fecha'][i],tz=bogota) for i in range(len(USD_feather))]

In [74]:
timestamp_base[0]

Timestamp('2022-01-03 08:30:00-0500', tz='America/Bogota')

In [75]:
USD_feather=USD_feather.set_index('Fecha')

In [76]:
USD_feather

,Hora,Demanda,Oferta
Fecha,,,
2022-01-03 08:00:24-05:00,08:00:24,NaN,4100.00
2022-01-03 08:00:24-05:00,08:00:24,3800.0,4100.00
2022-01-03 08:00:31-05:00,08:00:31,3850.0,4100.00
2022-01-03 08:00:35-05:00,08:00:35,3900.0,4100.00
2022-01-03 08:00:44-05:00,08:00:44,3901.0,4100.00
...,...,...,...
2022-05-12 12:59:45-05:00,12:59:45,4109.0,4109.95
2022-05-12 12:59:57-05:00,12:59:57,4108.6,4109.95
2022-05-12 12:59:57-05:00,12:59:57,4108.6,4109.98


In [77]:
#Eliminamos los registros duplicados y mantenemos la última observación para cada registro duplicado
_USD=USD_feather[~USD_feather.index.duplicated(keep='last')]

In [78]:
#Reindexamos para frecuencia de un minuto y llenamos los datos faltantes con estrategia forward fill
_USD=_USD.asfreq('1S',method='ffill')

In [79]:
_USD

,Hora,Demanda,Oferta
Fecha,,,
2022-01-03 08:00:24-05:00,08:00:24,3800.0,4100.00
2022-01-03 08:00:25-05:00,08:00:24,3800.0,4100.00
2022-01-03 08:00:26-05:00,08:00:24,3800.0,4100.00
2022-01-03 08:00:27-05:00,08:00:24,3800.0,4100.00
2022-01-03 08:00:28-05:00,08:00:24,3800.0,4100.00
...,...,...,...
2022-05-12 12:59:56-05:00,12:59:45,4109.0,4109.95
2022-05-12 12:59:57-05:00,12:59:57,4108.6,4109.98
2022-05-12 12:59:58-05:00,12:59:57,4108.6,4109.98


In [80]:
_USD['segundo']=[_USD.index[i].second for i in range(len(_USD))]

In [81]:
#Filtramos la base _USD por minuto 00
_USD=_USD[_USD['segundo']==0]

In [82]:
_USD=_USD.drop(['Hora','segundo'],axis=1)

In [83]:
_USD

,Demanda,Oferta
Fecha,,
2022-01-03 08:01:00-05:00,3950.0,4100.00
2022-01-03 08:02:00-05:00,3970.0,4099.00
2022-01-03 08:03:00-05:00,4000.0,4090.00
2022-01-03 08:04:00-05:00,4005.0,4089.50
2022-01-03 08:05:00-05:00,4005.0,4088.50
...,...,...
2022-05-12 12:56:00-05:00,4115.0,4116.00
2022-05-12 12:57:00-05:00,4112.5,4114.50
2022-05-12 12:58:00-05:00,4112.5,4113.98


In [84]:
#Renombramos las columnas
_USD.columns=['USD_BID','USD_ASK']

### Concatenar datos de variables núméricas para BID-ASK de las variables endógenas

In [85]:
#Creamos un dataframe con el histórico de los mejores precios disponibles en el libro de ofertas para las acciones
Depth=pd.DataFrame(columns=['Date','CIB_BID','CIB_ASK','PFB_BID','PFB_ASK'],index=timestamp_base)

In [86]:
#Agregamos las mejores disponibles en el libro de ofertas de la tasa de cambio
Depth=pd.concat([_USD,CIB_mid,PFB_mid],axis=1,join='outer')

In [87]:
Depth

,USD_BID,USD_ASK,ASK_CIB,BID_CIB,ASK_PFB,BID_PFB
2022-01-03 04:00:00-05:00,NaN,NaN,40.0,18.88,NaN,<NA>
2022-01-03 07:00:00-05:00,NaN,NaN,36.5,18.88,NaN,<NA>
2022-01-03 08:01:00-05:00,3950.0,4100.0,<NA>,<NA>,NaN,<NA>
2022-01-03 08:02:00-05:00,3970.0,4099.0,<NA>,<NA>,NaN,<NA>
2022-01-03 08:03:00-05:00,4000.0,4090.0,<NA>,<NA>,NaN,<NA>
...,...,...,...,...,...,...
2022-05-13 14:57:00-05:00,NaN,NaN,37.46,37.43,NaN,<NA>
2022-05-13 14:58:00-05:00,NaN,NaN,37.48,37.46,35090,39950
2022-05-13 14:59:00-05:00,NaN,NaN,37.5,37.49,38440,38330
2022-05-13 15:00:00-05:00,NaN,NaN,50.0,15.01,NaN,<NA>


### Concatenar variables categoricas y variables numéricas

In [88]:
#Concatenamos la base de mejores ofertas disponibles en libro con la base de los últimos precios operados y los features de tiempo.
base_total=pd.concat([base_cib,Depth],axis=1,join='outer')

In [89]:
base_total

,Date,Hora_COL,Minuto,Hora_NY,Festivo_COL,Festivo_NYSE,Next_Day,Subasta,PFB_ABIERTO,CIB_ABIERTO,...,PFB_close,next_day_close,SPX_close,tasa_cambio,USD_BID,USD_ASK,ASK_CIB,BID_CIB,ASK_PFB,BID_PFB
2022-01-03 04:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,40.0,18.88,NaN,<NA>
2022-01-03 07:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,36.5,18.88,NaN,<NA>
2022-01-03 08:01:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,NaN,3950.0,4100.0,<NA>,<NA>,NaN,<NA>
2022-01-03 08:02:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,NaN,3970.0,4099.0,<NA>,<NA>,NaN,<NA>
2022-01-03 08:03:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,NaN,4000.0,4090.0,<NA>,<NA>,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-13 16:57:00-05:00,2022-05-13,16.0,57.0,17,False,False,1,0,0,0,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,NaN,<NA>
2022-05-13 16:58:00-05:00,2022-05-13,16.0,58.0,17,False,False,1,0,0,0,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,NaN,<NA>
2022-05-13 16:59:00-05:00,2022-05-13,16.0,59.0,17,False,False,1,0,0,0,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,NaN,<NA>
2022-05-13 17:31:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,NaN,<NA>


In [90]:
base_total.index[186842].date()

datetime.date(2022, 5, 13)

In [91]:
#Agregamos la fecha a cada timestamp en el índice
base_total['Date']=[base_total.index[i].date() for i in range(len(base_total))]

In [92]:
base_total

,Date,Hora_COL,Minuto,Hora_NY,Festivo_COL,Festivo_NYSE,Next_Day,Subasta,PFB_ABIERTO,CIB_ABIERTO,...,PFB_close,next_day_close,SPX_close,tasa_cambio,USD_BID,USD_ASK,ASK_CIB,BID_CIB,ASK_PFB,BID_PFB
2022-01-03 04:00:00-05:00,2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,40.0,18.88,NaN,<NA>
2022-01-03 07:00:00-05:00,2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,36.5,18.88,NaN,<NA>
2022-01-03 08:01:00-05:00,2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,NaN,3950.0,4100.0,<NA>,<NA>,NaN,<NA>
2022-01-03 08:02:00-05:00,2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,NaN,3970.0,4099.0,<NA>,<NA>,NaN,<NA>
2022-01-03 08:03:00-05:00,2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,NaN,4000.0,4090.0,<NA>,<NA>,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-13 16:57:00-05:00,2022-05-13,16.0,57.0,17,False,False,1,0,0,0,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,NaN,<NA>
2022-05-13 16:58:00-05:00,2022-05-13,16.0,58.0,17,False,False,1,0,0,0,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,NaN,<NA>
2022-05-13 16:59:00-05:00,2022-05-13,16.0,59.0,17,False,False,1,0,0,0,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,NaN,<NA>
2022-05-13 17:31:00-05:00,2022-05-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,NaN,<NA>


In [93]:
#Filtramos por las fechas en las cuales ya teníamos datos en la base de precios de cierre (base_cib)
base_total=base_total[base_total.index.isin(base_cib.index)]

In [94]:
base_total

,Date,Hora_COL,Minuto,Hora_NY,Festivo_COL,Festivo_NYSE,Next_Day,Subasta,PFB_ABIERTO,CIB_ABIERTO,...,PFB_close,next_day_close,SPX_close,tasa_cambio,USD_BID,USD_ASK,ASK_CIB,BID_CIB,ASK_PFB,BID_PFB
2022-01-03 08:22:00-05:00,2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,NaN,4030.0,4069.0,<NA>,<NA>,NaN,<NA>
2022-01-03 08:30:00-05:00,2022-01-03,8.0,30.0,8,False,False,0,0,0,0,...,<NA>,<NA>,<NA>,NaN,4030.0,4050.0,<NA>,<NA>,NaN,<NA>
2022-01-03 08:31:00-05:00,2022-01-03,8.0,31.0,8,False,False,0,0,0,0,...,<NA>,<NA>,<NA>,NaN,4030.0,4050.0,<NA>,<NA>,NaN,<NA>
2022-01-03 08:32:00-05:00,2022-01-03,8.0,32.0,8,False,False,0,0,0,0,...,<NA>,<NA>,<NA>,NaN,4030.0,4050.0,<NA>,<NA>,NaN,<NA>
2022-01-03 08:33:00-05:00,2022-01-03,8.0,33.0,8,False,False,0,0,0,0,...,<NA>,<NA>,<NA>,NaN,4030.0,4050.0,<NA>,<NA>,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-13 16:57:00-05:00,2022-05-13,16.0,57.0,17,False,False,1,0,0,0,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,NaN,<NA>
2022-05-13 16:58:00-05:00,2022-05-13,16.0,58.0,17,False,False,1,0,0,0,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,NaN,<NA>
2022-05-13 16:59:00-05:00,2022-05-13,16.0,59.0,17,False,False,1,0,0,0,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,NaN,<NA>
2022-05-13 17:31:00-05:00,2022-05-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>,<NA>,NaN,<NA>


### Preprocesamiento: Datos faltantes

In [95]:
base_total.columns

Index(['Date', 'Hora_COL', 'Minuto', 'Hora_NY', 'Festivo_COL', 'Festivo_NYSE',
       'Next_Day', 'Subasta', 'PFB_ABIERTO', 'CIB_ABIERTO', 'CIB_close',
       'COP_close', 'PFB_close', 'next_day_close', 'SPX_close', 'tasa_cambio',
       'USD_BID', 'USD_ASK', 'ASK_CIB', 'BID_CIB', 'ASK_PFB', 'BID_PFB'],
      dtype='object')

In [96]:
#Definimos el conjunto de columnas numéricas: son columnas que contienen información de los precios
columnas_precios=['CIB_close',
       'COP_close', 'PFB_close', 'next_day_close', 'SPX_close', 'tasa_cambio',
       'USD_BID', 'USD_ASK', 'ASK_CIB', 'BID_CIB', 'ASK_PFB', 'BID_PFB']

In [97]:
#realizamos una estrategia de llenado forward fill para las columnas que contienen información de precios
for columna in columnas_precios:
    base_total[columna]=base_total[columna].ffill()

C:\Users\fract\AppData\Local\Temp\ipykernel_21248\487560459.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_total[columna]=base_total[columna].ffill()


In [98]:
base_total

,Date,Hora_COL,Minuto,Hora_NY,Festivo_COL,Festivo_NYSE,Next_Day,Subasta,PFB_ABIERTO,CIB_ABIERTO,...,PFB_close,next_day_close,SPX_close,tasa_cambio,USD_BID,USD_ASK,ASK_CIB,BID_CIB,ASK_PFB,BID_PFB
2022-01-03 08:22:00-05:00,2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,NaN,4030.0,4069.0,<NA>,<NA>,NaN,<NA>
2022-01-03 08:30:00-05:00,2022-01-03,8.0,30.0,8,False,False,0,0,0,0,...,<NA>,<NA>,<NA>,NaN,4030.0,4050.0,<NA>,<NA>,NaN,<NA>
2022-01-03 08:31:00-05:00,2022-01-03,8.0,31.0,8,False,False,0,0,0,0,...,<NA>,<NA>,<NA>,NaN,4030.0,4050.0,<NA>,<NA>,NaN,<NA>
2022-01-03 08:32:00-05:00,2022-01-03,8.0,32.0,8,False,False,0,0,0,0,...,<NA>,<NA>,<NA>,NaN,4030.0,4050.0,<NA>,<NA>,NaN,<NA>
2022-01-03 08:33:00-05:00,2022-01-03,8.0,33.0,8,False,False,0,0,0,0,...,<NA>,<NA>,<NA>,NaN,4030.0,4050.0,<NA>,<NA>,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-13 16:57:00-05:00,2022-05-13,16.0,57.0,17,False,False,1,0,0,0,...,38440,4100.0,32196.66,4100.0,4108.1,4110.0,50.0,0.0,38440.0,38330
2022-05-13 16:58:00-05:00,2022-05-13,16.0,58.0,17,False,False,1,0,0,0,...,38440,4100.0,32196.66,4100.0,4108.1,4110.0,50.0,0.0,38440.0,38330
2022-05-13 16:59:00-05:00,2022-05-13,16.0,59.0,17,False,False,1,0,0,0,...,38440,4100.0,32196.66,4100.0,4108.1,4110.0,50.0,0.0,38440.0,38330
2022-05-13 17:31:00-05:00,2022-05-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38440,4100.0,32196.66,4100.0,4108.1,4110.0,50.0,0.0,38440.0,38330


In [99]:
#Buscamos los datos perdidos
base_total['PFB_close'].isnull().sum()

85

In [100]:
#Eliminamos los datos perdidos teniendo en cuenta que son los primeros registros de la base
BIG=base_total[base_total['PFB_close'].isnull()==False]

In [101]:
BIG

,Date,Hora_COL,Minuto,Hora_NY,Festivo_COL,Festivo_NYSE,Next_Day,Subasta,PFB_ABIERTO,CIB_ABIERTO,...,PFB_close,next_day_close,SPX_close,tasa_cambio,USD_BID,USD_ASK,ASK_CIB,BID_CIB,ASK_PFB,BID_PFB
2022-01-03 09:54:00-05:00,2022-01-03,9.0,54.0,9,False,False,0,0,1,1,...,32000,<NA>,36351.3,4075.0,4073.00,4076.9,31.74,31.55,32300.0,32010
2022-01-03 09:55:00-05:00,2022-01-03,9.0,55.0,9,False,False,0,0,1,1,...,32000,<NA>,36357.37,4075.0,4072.72,4076.0,31.75,31.58,32300.0,32290
2022-01-03 09:56:00-05:00,2022-01-03,9.0,56.0,9,False,False,0,0,1,1,...,32300,<NA>,36366.59,4075.0,4072.72,4076.0,31.76,31.6,32300.0,32290
2022-01-03 09:57:00-05:00,2022-01-03,9.0,57.0,9,False,False,0,0,1,1,...,32300,<NA>,36389.77,4073.86,4072.72,4075.0,31.76,31.6,32300.0,32290
2022-01-03 09:58:00-05:00,2022-01-03,9.0,58.0,9,False,False,0,0,1,1,...,32300,<NA>,36389.36,4072.72,4072.00,4075.0,31.76,31.62,32300.0,32290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-13 16:57:00-05:00,2022-05-13,16.0,57.0,17,False,False,1,0,0,0,...,38440,4100.0,32196.66,4100.0,4108.10,4110.0,50.0,0.0,38440.0,38330
2022-05-13 16:58:00-05:00,2022-05-13,16.0,58.0,17,False,False,1,0,0,0,...,38440,4100.0,32196.66,4100.0,4108.10,4110.0,50.0,0.0,38440.0,38330
2022-05-13 16:59:00-05:00,2022-05-13,16.0,59.0,17,False,False,1,0,0,0,...,38440,4100.0,32196.66,4100.0,4108.10,4110.0,50.0,0.0,38440.0,38330
2022-05-13 17:31:00-05:00,2022-05-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38440,4100.0,32196.66,4100.0,4108.10,4110.0,50.0,0.0,38440.0,38330


In [102]:
BIG.isnull().any()

Date              False
Hora_COL           True
Minuto             True
Hora_NY            True
Festivo_COL        True
Festivo_NYSE       True
Next_Day           True
Subasta            True
PFB_ABIERTO        True
CIB_ABIERTO        True
CIB_close         False
COP_close         False
PFB_close         False
next_day_close     True
SPX_close         False
tasa_cambio       False
USD_BID           False
USD_ASK           False
ASK_CIB           False
BID_CIB           False
ASK_PFB           False
BID_PFB           False
dtype: bool

In [103]:
#filtramos los registros que no contienen información acerca de la hora
BIG=BIG.drop(BIG.loc[BIG['Hora_COL'].isnull()==True].index)

In [104]:
#eliminamos la columna next day debido a que la tasa de cambkio ya contiene la información asociada a esta columna
BIG=BIG.drop(columns=['next_day_close'],axis=1)

In [105]:
#observamos que con la estrategia de preprocesamiento utilizada ya no tenemos datos perdidos
BIG.isnull().any()

Date            False
Hora_COL        False
Minuto          False
Hora_NY         False
Festivo_COL     False
Festivo_NYSE    False
Next_Day        False
Subasta         False
PFB_ABIERTO     False
CIB_ABIERTO     False
CIB_close       False
COP_close       False
PFB_close       False
SPX_close       False
tasa_cambio     False
USD_BID         False
USD_ASK         False
ASK_CIB         False
BID_CIB         False
ASK_PFB         False
BID_PFB         False
dtype: bool

### Columnas de Spread entre los precios del mismo activo en diferentes mercados

In [106]:
#Agregamos las columnas Spread_BE_SL, Spread_BL_SE, MID_Spread_LE, MID_Spread_EL
swap=0.5 #Definimos un costo constante de swap de 0.5 para las operaciones en divisa
factor=4 #Definimos el factor de conversión para los ADR
costo=0.008 #Definimos el costo transaccional para las operaciones de ADR en el exterior.
BIG['Spread_BE_SL']=None
BIG['Spread_BL_SE']=None
for i in range(len(BIG)):
        externo_buy=(BIG['ASK_CIB'].iloc[i]+costo)*(BIG['USD_ASK'].iloc[i]+swap)/factor
        externo_sell=(BIG['BID_CIB'].iloc[i]-costo)*(BIG['USD_BID'].iloc[i]-swap)/factor
        BIG['Spread_BE_SL'].iloc[i]=BIG['BID_PFB'].iloc[i]-externo_buy
        BIG['Spread_BL_SE'].iloc[i]=externo_sell-BIG['ASK_PFB'].iloc[i]

C:\Users\fract\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [107]:
base_all=BIG.reset_index()

In [108]:
base_all=base_all.drop(['index'],axis=1)

In [109]:
base_all

,Date,Hora_COL,Minuto,Hora_NY,Festivo_COL,Festivo_NYSE,Next_Day,Subasta,PFB_ABIERTO,CIB_ABIERTO,...,SPX_close,tasa_cambio,USD_BID,USD_ASK,ASK_CIB,BID_CIB,ASK_PFB,BID_PFB,Spread_BE_SL,Spread_BL_SE
0,2022-01-03,9.0,54.0,9,False,False,0,0,1,1,...,36351.3,4075.0,4073.00,4076.9,31.74,31.55,32300.0,32010,-352.3238,-186.30125
1,2022-01-03,9.0,55.0,9,False,False,0,0,1,1,...,36357.37,4075.0,4072.72,4076.0,31.75,31.58,32300.0,32290,-75.37175,-157.96754
2,2022-01-03,9.0,56.0,9,False,False,0,0,1,1,...,36366.59,4075.0,4072.72,4076.0,31.76,31.6,32300.0,32290,-85.563,-137.60644
3,2022-01-03,9.0,57.0,9,False,False,0,0,1,1,...,36389.77,4073.86,4072.72,4075.0,31.76,31.6,32300.0,32290,-77.621,-137.60644
4,2022-01-03,9.0,58.0,9,False,False,0,0,1,1,...,36389.36,4072.72,4072.00,4075.0,31.76,31.62,32300.0,32290,-77.621,-122.9355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51181,2022-05-13,16.0,55.0,17,False,False,1,0,0,0,...,32196.66,4100.0,4108.10,4110.0,50.0,0.0,38440.0,38330,-13059.471,-38448.2152
51182,2022-05-13,16.0,56.0,17,False,False,1,0,0,0,...,32196.66,4100.0,4108.10,4110.0,50.0,0.0,38440.0,38330,-13059.471,-38448.2152
51183,2022-05-13,16.0,57.0,17,False,False,1,0,0,0,...,32196.66,4100.0,4108.10,4110.0,50.0,0.0,38440.0,38330,-13059.471,-38448.2152
51184,2022-05-13,16.0,58.0,17,False,False,1,0,0,0,...,32196.66,4100.0,4108.10,4110.0,50.0,0.0,38440.0,38330,-13059.471,-38448.2152


In [110]:
base_all['Date']=pd.to_datetime(base_all['Date'], format='%Y-%m-%d')

In [111]:
#Filtramos las fechas mayores a la información disponible en el USD.feather
base_all=base_all.drop(base_all[(base_all['Date']=='2022-05-12') &
                                (base_all['Hora_COL']>=13) | 
                                (base_all['Date']=='2022-05-13')].index)

In [112]:
base_all

,Date,Hora_COL,Minuto,Hora_NY,Festivo_COL,Festivo_NYSE,Next_Day,Subasta,PFB_ABIERTO,CIB_ABIERTO,...,SPX_close,tasa_cambio,USD_BID,USD_ASK,ASK_CIB,BID_CIB,ASK_PFB,BID_PFB,Spread_BE_SL,Spread_BL_SE
0,2022-01-03,9.0,54.0,9,False,False,0,0,1,1,...,36351.3,4075.0,4073.00,4076.90,31.74,31.55,32300.0,32010,-352.3238,-186.30125
1,2022-01-03,9.0,55.0,9,False,False,0,0,1,1,...,36357.37,4075.0,4072.72,4076.00,31.75,31.58,32300.0,32290,-75.37175,-157.96754
2,2022-01-03,9.0,56.0,9,False,False,0,0,1,1,...,36366.59,4075.0,4072.72,4076.00,31.76,31.6,32300.0,32290,-85.563,-137.60644
3,2022-01-03,9.0,57.0,9,False,False,0,0,1,1,...,36389.77,4073.86,4072.72,4075.00,31.76,31.6,32300.0,32290,-77.621,-137.60644
4,2022-01-03,9.0,58.0,9,False,False,0,0,1,1,...,36389.36,4072.72,4072.00,4075.00,31.76,31.62,32300.0,32290,-77.621,-122.9355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50401,2022-05-12,12.0,55.0,13,False,False,0,0,1,1,...,31349.44,4115.25,4114.50,4116.00,36.65,36.56,37690.0,37610,-115.66425,-96.268
50402,2022-05-12,12.0,56.0,13,False,False,0,0,1,1,...,31344.96,4115.25,4115.00,4116.00,36.65,36.56,37680.0,37610,-115.66425,-81.699
50403,2022-05-12,12.0,57.0,13,False,False,0,0,1,1,...,31362.22,4114.0,4112.50,4114.50,36.67,36.6,37680.0,37610,-122.4925,-63.424
50404,2022-05-12,12.0,58.0,13,False,False,0,0,1,1,...,31408.44,4112.5,4112.50,4113.98,36.69,36.59,37680.0,37610,-138.29676,-73.704


In [113]:
#Enviamos a excel
base_all.to_excel("base_cib.xlsx")

In [114]:
base_all.columns

Index(['Date', 'Hora_COL', 'Minuto', 'Hora_NY', 'Festivo_COL', 'Festivo_NYSE',
       'Next_Day', 'Subasta', 'PFB_ABIERTO', 'CIB_ABIERTO', 'CIB_close',
       'COP_close', 'PFB_close', 'SPX_close', 'tasa_cambio', 'USD_BID',
       'USD_ASK', 'ASK_CIB', 'BID_CIB', 'ASK_PFB', 'BID_PFB', 'Spread_BE_SL',
       'Spread_BL_SE'],
      dtype='object')

In [115]:
columnas_precio=['CIB_close','COP_close', 'PFB_close', 'SPX_close', 'tasa_cambio', 'USD_BID',
       'USD_ASK', 'ASK_CIB', 'BID_CIB', 'ASK_PFB', 'BID_PFB', 'Spread_BE_SL',
       'Spread_BL_SE']

In [116]:
len(base_all)

50406

### Función de Retrasos/Definición de Arbitraje

In [117]:
#Creamos un data frame de retrasos para cada instrumento
def add_lags(data, ric, lags):
    cols = []
    df = pd.DataFrame(data[ric])
    for lag in range(1, lags + 1):
        col = 'lag_{}'.format(lag)+'_'+str(ric)  # defines the column name
        df[col] = df[ric].shift(lag)  # creates the lagged data column
        cols.append(col)  # stores the column name
    df.dropna(inplace=True)  # gets rid of incomplete data rows
    return df, cols

In [118]:
#Observamos los lagas para una de las features de nuestra base de datos
add_lags(base_all,'CIB_close',2)[0]

,CIB_close,lag_1_CIB_close,lag_2_CIB_close
2,31.6697,31.58,31.58
3,31.73,31.6697,31.58
4,31.68,31.73,31.6697
5,31.68,31.68,31.73
6,31.62,31.68,31.68
...,...,...,...
50401,36.628,36.65,36.63
50402,36.61,36.628,36.65
50403,36.56,36.61,36.628
50404,36.67,36.56,36.61


In [119]:
columnas_precio[1:]

['COP_close',
 'PFB_close',
 'SPX_close',
 'tasa_cambio',
 'USD_BID',
 'USD_ASK',
 'ASK_CIB',
 'BID_CIB',
 'ASK_PFB',
 'BID_PFB',
 'Spread_BE_SL',
 'Spread_BL_SE']

In [120]:
#Crear base Lags
def hist_lags(base,lags):
    base_lags=add_lags(base,columnas_precio[0],lags)[0].iloc[:,1:]
    for column in columnas_precio[1:]:
        bse=add_lags(base,column,lags)[0].iloc[:,1:]
        base_lags=pd.concat([base_lags,bse],axis=1,join='outer')
    return base_lags
    

In [121]:
#Precios con dos rezagos
hist_lags(base_all, 2)

,lag_1_CIB_close,lag_2_CIB_close,lag_1_COP_close,lag_2_COP_close,lag_1_PFB_close,lag_2_PFB_close,lag_1_SPX_close,lag_2_SPX_close,lag_1_tasa_cambio,lag_2_tasa_cambio,...,lag_1_BID_CIB,lag_2_BID_CIB,lag_1_ASK_PFB,lag_2_ASK_PFB,lag_1_BID_PFB,lag_2_BID_PFB,lag_1_Spread_BE_SL,lag_2_Spread_BE_SL,lag_1_Spread_BL_SE,lag_2_Spread_BL_SE
2,31.58,31.58,4075.0,4075.0,32000,32000,36357.37,36351.3,4075.0,4075.0,...,31.58,31.55,32300.0,32300.0,32290,32010,-75.37175,-352.3238,-157.96754,-186.30125
3,31.6697,31.58,4075.0,4075.0,32300,32000,36366.59,36357.37,4075.0,4075.0,...,31.6,31.58,32300.0,32300.0,32290,32290,-85.563,-75.37175,-137.60644,-157.96754
4,31.73,31.6697,4073.86,4075.0,32300,32300,36389.77,36366.59,4073.86,4075.0,...,31.6,31.6,32300.0,32300.0,32290,32290,-77.621,-85.563,-137.60644,-137.60644
5,31.68,31.73,4072.72,4073.86,32300,32300,36389.36,36389.77,4072.72,4073.86,...,31.62,31.6,32300.0,32300.0,32290,32290,-77.621,-77.621,-122.9355,-137.60644
6,31.68,31.68,4072.0,4072.72,32300,32300,36356.86,36389.36,4072.0,4072.72,...,31.62,31.62,32300.0,32300.0,32290,32290,-69.2819,-77.621,-122.9355,-122.9355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50401,36.65,36.63,4114.6,4114.8,37600,37670,31357.45,31337.51,4114.6,4114.8,...,36.61,36.62,37690.0,37690.0,37610,37610,-124.76379,-125.24335,-47.130625,-44.6263
50402,36.628,36.65,4115.25,4114.6,37600,37600,31349.44,31357.45,4115.25,4114.6,...,36.56,36.61,37690.0,37690.0,37610,37610,-115.66425,-124.76379,-96.268,-47.130625
50403,36.61,36.628,4115.25,4115.25,37600,37600,31344.96,31349.44,4115.25,4115.25,...,36.56,36.56,37680.0,37690.0,37610,37610,-115.66425,-115.66425,-81.699,-96.268
50404,36.56,36.61,4114.0,4115.25,37600,37600,31362.22,31344.96,4114.0,4115.25,...,36.6,36.56,37680.0,37680.0,37610,37610,-122.4925,-115.66425,-63.424,-81.699


In [122]:
#Funcion para expandir la base con algún número de rezagos
def expandir_base(data,columna,lags):
    dfs={}
    df = add_lags(data, columna, lags)[0]
    dfs[columna] = df
    new_data=pd.concat([data.iloc[lags:,:],df.iloc[:,1:]],axis=1)
    return new_data
        

In [123]:
expandir_base(base_all,'CIB_close',2)

,Date,Hora_COL,Minuto,Hora_NY,Festivo_COL,Festivo_NYSE,Next_Day,Subasta,PFB_ABIERTO,CIB_ABIERTO,...,USD_BID,USD_ASK,ASK_CIB,BID_CIB,ASK_PFB,BID_PFB,Spread_BE_SL,Spread_BL_SE,lag_1_CIB_close,lag_2_CIB_close
2,2022-01-03,9.0,56.0,9,False,False,0,0,1,1,...,4072.72,4076.00,31.76,31.6,32300.0,32290,-85.563,-137.60644,31.58,31.58
3,2022-01-03,9.0,57.0,9,False,False,0,0,1,1,...,4072.72,4075.00,31.76,31.6,32300.0,32290,-77.621,-137.60644,31.6697,31.58
4,2022-01-03,9.0,58.0,9,False,False,0,0,1,1,...,4072.00,4075.00,31.76,31.62,32300.0,32290,-77.621,-122.9355,31.73,31.6697
5,2022-01-03,9.0,59.0,9,False,False,0,0,1,1,...,4072.00,4073.95,31.76,31.62,32300.0,32290,-69.2819,-122.9355,31.68,31.73
6,2022-01-03,10.0,0.0,10,False,False,0,0,1,1,...,4070.50,4073.00,31.76,31.66,32300.0,32290,-61.737,-94.09,31.68,31.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50401,2022-05-12,12.0,55.0,13,False,False,0,0,1,1,...,4114.50,4116.00,36.65,36.56,37690.0,37610,-115.66425,-96.268,36.65,36.63
50402,2022-05-12,12.0,56.0,13,False,False,0,0,1,1,...,4115.00,4116.00,36.65,36.56,37680.0,37610,-115.66425,-81.699,36.628,36.65
50403,2022-05-12,12.0,57.0,13,False,False,0,0,1,1,...,4112.50,4114.50,36.67,36.6,37680.0,37610,-122.4925,-63.424,36.61,36.628
50404,2022-05-12,12.0,58.0,13,False,False,0,0,1,1,...,4112.50,4113.98,36.69,36.59,37680.0,37610,-138.29676,-73.704,36.56,36.61


In [124]:
#Crear base Lags para todos los intrumentos
def base_lags(base,lags):
    base_lags=add_lags(base_all,columnas_precio[0],lags)[0].iloc[:,1:]
    for column in columnas_precio[1:]:
        bse=add_lags(base_all,column,lags)[0].iloc[:,1:]
        base_lags=pd.concat([base_lags,bse],axis=1,join='outer')
    return pd.concat([base.iloc[lags:,:],base_lags],axis=1,join='outer')
    

In [125]:
base_lags(base_all,2)

,Date,Hora_COL,Minuto,Hora_NY,Festivo_COL,Festivo_NYSE,Next_Day,Subasta,PFB_ABIERTO,CIB_ABIERTO,...,lag_1_BID_CIB,lag_2_BID_CIB,lag_1_ASK_PFB,lag_2_ASK_PFB,lag_1_BID_PFB,lag_2_BID_PFB,lag_1_Spread_BE_SL,lag_2_Spread_BE_SL,lag_1_Spread_BL_SE,lag_2_Spread_BL_SE
2,2022-01-03,9.0,56.0,9,False,False,0,0,1,1,...,31.58,31.55,32300.0,32300.0,32290,32010,-75.37175,-352.3238,-157.96754,-186.30125
3,2022-01-03,9.0,57.0,9,False,False,0,0,1,1,...,31.6,31.58,32300.0,32300.0,32290,32290,-85.563,-75.37175,-137.60644,-157.96754
4,2022-01-03,9.0,58.0,9,False,False,0,0,1,1,...,31.6,31.6,32300.0,32300.0,32290,32290,-77.621,-85.563,-137.60644,-137.60644
5,2022-01-03,9.0,59.0,9,False,False,0,0,1,1,...,31.62,31.6,32300.0,32300.0,32290,32290,-77.621,-77.621,-122.9355,-137.60644
6,2022-01-03,10.0,0.0,10,False,False,0,0,1,1,...,31.62,31.62,32300.0,32300.0,32290,32290,-69.2819,-77.621,-122.9355,-122.9355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50401,2022-05-12,12.0,55.0,13,False,False,0,0,1,1,...,36.61,36.62,37690.0,37690.0,37610,37610,-124.76379,-125.24335,-47.130625,-44.6263
50402,2022-05-12,12.0,56.0,13,False,False,0,0,1,1,...,36.56,36.61,37690.0,37690.0,37610,37610,-115.66425,-124.76379,-96.268,-47.130625
50403,2022-05-12,12.0,57.0,13,False,False,0,0,1,1,...,36.56,36.56,37680.0,37690.0,37610,37610,-115.66425,-115.66425,-81.699,-96.268
50404,2022-05-12,12.0,58.0,13,False,False,0,0,1,1,...,36.6,36.56,37680.0,37680.0,37610,37610,-122.4925,-115.66425,-63.424,-81.699


### Función de Arbitraje

Teniendo en cuenta las características particulares y las condiciones de tiempo en cada mercado, definiremos una función de arbitraje de la siguiente forma:  
- Si es festivo en ambos mercados o si ambos mercados están cerrados, diremos que no hay arbitraje.
- En caso contrario, si ambos mercados están abiertos y no es horario de subasta: hay arbitraje cuando el spread de la diferencia en precios de las cotizaciones es mayor que un determinado margen.  
- Ahora bien, si ambos mercados están abiertos pero hay horario de subasta en BVC, es decir, no hay posibilidad de calce automático en Colombia, pero se pueden enviar órdenes para participar en la subasta, diremos que se presentó arbitraje si al comparar los precios de cierre del mercado local (el precio de cierre de la subasta) y el precio de cierre del mercado externo, el spread es superior al margen.  
- Si el mercado en Colombia está cerrado, dieremos que se presentó arbitraje cuando al comparar los últimos precios de cierre en Colombia, para las acciones y la tasa de cambio, contra el precio actual en el mercado externo, se supera el spread.  
- Si el mercado en NY está cerrado, diremos que se presentó arbitraje cuando al comparar los precios actuales en el mercado local colombiano contra el último precio de cierre de la acción en la bolsa de NY, se supera el spread:



In [126]:
#Arbitraje directo
def Arbitraje(base,margen):
    #local=precio local
    #externo=precio externo
    #lag=retraso
    #costo=fee de operación
    #margen=margen esperado de arbitraje
    #swap=costo de swap financiero en pesos por usd
    base['Arbitraje_BE_SL']=None
    base['Arbitraje_BL_SE']=None
    #Llenamos la columna arbitraje buy externo/sell local
    for i in range(len(base)):
        #si hay festivo en ambos mercados, no hay arbitraje
        if (base['Festivo_COL'].iloc[i]==True) and (base['Festivo_NYSE'].iloc[i]==True) : 
            base['Arbitraje_BE_SL'].iloc[i]=0
        #si ambos mercados están cerrados por horario, no hay arbitraje
        elif (base['PFB_ABIERTO'].iloc[i]==0) and (base['CIB_ABIERTO'].iloc[i]==0) : 
            base['Arbitraje_BE_SL'].iloc[i]=0
        #si ambos mercados están abiertos y no es subasta de cierre, comparamos los spreads
        elif (base['PFB_ABIERTO'].iloc[i]==1) and (base['CIB_ABIERTO'].iloc[i]==1) and (base['Subasta'].iloc[i]==0): #ambos abiertos en sesion mercado abierto
            if (base['Spread_BE_SL'].iloc[i])>=margen: #el spread bid/ask debe ser mayor que el margen
                base['Arbitraje_BE_SL'].iloc[i]=1
            else:
                base['Arbitraje_BE_SL'].iloc[i]=0
        #Si ambos mercados están abiertos y es subasta de cierre
        elif (base['PFB_ABIERTO'].iloc[i]==1) and (base['CIB_ABIERTO'].iloc[i]==1) and (base['Subasta'].iloc[i]==1): #en la subasta de cierre buscamos el precio de cierre de PFB y lo comparamos con CIB
            day=base['Date'].iloc[i] #Buscamos el día
            hora=base['Hora_COL'].iloc[i] #Buscamos la hora colombia
            hora_ny=base['Hora_NY'].iloc[i] #Buscamos la hora ny
            if hora==hora_ny: #Si el horario de ambos mercados coincide
                #buscamos el precio de cierre de la acción en la subasta 16:01 y lo comparamos con el precio externo
                close_PFB=float(base[(base['Date']==day) & (base['Hora_COL']==16) & (base['Minuto']==1)]['PFB_close'])
                cib_eq=(base['ASK_CIB'].iloc[i]+0.008)*(base['USD_ASK'].iloc[i]+0.5)/4
            if hora!=hora_ny:#Si el horario de ambos mercados es distinto
                #buscamos el precio de cierre de la acción a las 15:01 y lo comparamos con el precio externo
                close_PFB=float(base[(base['Date']==day) & (base['Hora_COL']==15) & (base['Minuto']==1)]['PFB_close'])
                cib_eq=(base['ASK_CIB'].iloc[i]+0.008)*(base['USD_ASK'].iloc[i]+0.5)/4
            if close_PFB-cib_eq>=margen: # Si hay margen con respecto al precio de cierre en la subasta
                base['Arbitraje_BE_SL'].iloc[i]=1
            else:
                base['Arbitraje_BE_SL'].iloc[i]=0
        #si Colombia está cerrado pero NY abierto, la referencia del precio en Colombia es el close.
        elif (base['PFB_ABIERTO'].iloc[i]==0) and (base['CIB_ABIERTO'].iloc[i]==1): 
            close_PFB=base['PFB_close'].iloc[i]
            usd_ask=base['COP_close'].iloc[i]
            cib_eq=(base['ASK_CIB'].iloc[i]+0.008)*(usd_ask+0.5)/4
            if close_PFB-cib_eq>=margen:
                base['Arbitraje_BE_SL'].iloc[i]=1
            else:
                base['Arbitraje_BE_SL'].iloc[i]=0
        #si NY está cerrado pero Colombia abierto, la referencia del precio en NY es el close
        elif (base['CIB_ABIERTO'].iloc[i]==0) and (base['PFB_ABIERTO'].iloc[i]==1) and (base['Subasta'].iloc[i]==0): 
            close_CIB=base['CIB_close'].iloc[i] #buscamos el último precio de CIB y lo compraramos
            cib_eq=(close_CIB+0.008)*(base['USD_ASK'].iloc[i]+0.5)/4
            if base['BID_PFB'].iloc[i]-cib_eq>=margen:
                base['Arbitraje_BE_SL'].iloc[i]=1
            else:
                base['Arbitraje_BE_SL'].iloc[i]=0
        elif (base['CIB_ABIERTO'].iloc[i]==0) and (base['PFB_ABIERTO'].iloc[i]==1) and (base['Subasta'].iloc[i]==1): # en la subasta de cierre, buscamos el cierre
            day=base['Date'].iloc[i] #Buscamos el día
            hora=base['Hora_COL'].iloc[i] #Buscamos la hora colombia
            hora_ny=base['Hora_NY'].iloc[i] #Buscamos la hora ny
            if hora==hora_ny: #Si el horario de ambos mercados coincide
                #buscamos el precio de cierre de la acción en la subasta 16:01 y lo comparamos con el precio externo
                close_PFB=float(base[(base['Date']==day) & (base['Hora_COL']==16) & (base['Minuto']==1)]['PFB_close'])
                if close_PFB-cib_eq >= margen:
                    base['Arbitraje_BE_SL'].iloc[i]=1
                else:
                    base['Arbitraje_BE_SL'].iloc[i]=0
            if hora!=hora_ny:#Si el horario de ambos mercados es distinto
            #buscamos el precio de cierre de la acción a las 15:01 y lo comparamos con el precio externo
                close_PFB=float(base[(base['Date']==day) & (base['Hora_COL']==15) & (base['Minuto']==1)]['PFB_close'])
                if close_PFB-cib_eq>=margen:
                    base['Arbitraje_BE_SL'].iloc[i]=1
                else:
                    base['Arbitraje_BE_SL'].iloc[i]=0
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------                                       
            
    #Llenamos la columna Buylocal/SellExterno            
    for i in range(len(base)):
        if (base['Festivo_COL'].iloc[i]==True) and (base['Festivo_NYSE'].iloc[i]==True) : #si hay festivo en ambos mercados
            base['Arbitraje_BL_SE'].iloc[i]=0
        
        elif (base['PFB_ABIERTO'].iloc[i]==0) and (base['CIB_ABIERTO'].iloc[i]==0): #si ambos mercados están cerrados
            base['Arbitraje_BL_SE'].iloc[i]=0
        elif (base['PFB_ABIERTO'].iloc[i]==1) and (base['CIB_ABIERTO'].iloc[i]==1) and (base['Subasta'].iloc[i]==0): #ambos abiertos en sesion mercado abierto
            if (base['Spread_BL_SE'].iloc[i])>=margen: #el spread bid/ask debe ser mayor que el margen
                base['Arbitraje_BL_SE'].iloc[i]=1
            else:
                base['Arbitraje_BL_SE'].iloc[i]=0
        elif (base['PFB_ABIERTO'].iloc[i]==1) and (base['CIB_ABIERTO'].iloc[i]==1) and (base['Subasta'].iloc[i]==1): #en la subasta de cierre buscamos el precio de cierre de PFB y lo comparamos con CIB
            day=base['Date'].iloc[i]
            hora=base['Hora_COL'].iloc[i]
            hora_ny=base['Hora_NY'].iloc[i]
            if hora==hora_ny:
                close_PFB=float(base[(base['Date']==day) & (base['Hora_COL']==16) & (base['Minuto']==1)]['PFB_close'])
                cib_eq=(base['BID_CIB'].iloc[i]-0.008)*(base['USD_BID'].iloc[i]-0.5)/4
            if hora!=hora_ny:
                close_PFB=float(base[(base['Date']==day) & (base['Hora_COL']==15) & (base['Minuto']==1)]['PFB_close'])
                cib_eq=(base['BID_CIB'].iloc[i]-0.008)*(base['USD_BID'].iloc[i]-0.5)/4
            if cib_eq-close_PFB>=margen:
                base['Arbitraje_BL_SE'].iloc[i]=1
            else:
                base['Arbitraje_BL_SE'].iloc[i]=0
        
        elif (base['PFB_ABIERTO'].iloc[i]==0) and (base['CIB_ABIERTO'].iloc[i]==1): #si Colombia está cerrado pero NY abierto
            close_PFB=base['PFB_close'].iloc[i]
            usd_bid=base['COP_close'].iloc[i]
            cib_eq=(base['BID_CIB'].iloc[i]-0.008)*(usd_bid-0.5)/4
            if cib_eq-close_PFB>=margen:
                base['Arbitraje_BL_SE'].iloc[i]=1
            else:
                base['Arbitraje_BL_SE'].iloc[i]=0
                
        
                
        elif (base['CIB_ABIERTO'].iloc[i]==0) and (base['PFB_ABIERTO'].iloc[i]==1) and (base['Subasta'].iloc[i]==0): #si NY está cerrado pero Colombia abierto
            close_CIB=base['CIB_close'].iloc[i] #buscamos el último precio de CIB y lo compraramos
            cib_eq=(close_CIB-0.008)*(base['USD_BID'].iloc[i]-0.5)/4
            if cib_eq-base['ASK_PFB'].iloc[i]>=margen:
                base['Arbitraje_BL_SE'].iloc[i]=1
            else:
                base['Arbitraje_BL_SE'].iloc[i]=0
        elif (base['CIB_ABIERTO'].iloc[i]==0) and (base['PFB_ABIERTO'].iloc[i]==1) and (base['Subasta'].iloc[i]==1): # en la subasta de cierre, buscamos el cierre
            day=base['Date'].iloc[i] #Buscamos el día
            hora=base['Hora_COL'].iloc[i] #Buscamos la hora colombia
            hora_ny=base['Hora_NY'].iloc[i] #Buscamos la hora ny
            if hora==hora_ny: #Si el horario de ambos mercados coincide
                #buscamos el precio de cierre de la acción en la subasta 16:01 y lo comparamos con el precio externo
                close_PFB=float(base[(base['Date']==day) & (base['Hora_COL']==16) & (base['Minuto']==1)]['PFB_close'])
                if cib_eq-close_PFB >= margen:
                    base['Arbitraje_BL_SE'].iloc[i]=1
                else:
                    base['Arbitraje_BL_SE'].iloc[i]=0
            if hora!=hora_ny:#Si el horario de ambos mercados es distinto
            #buscamos el precio de cierre de la acción a las 15:01 y lo comparamos con el precio externo
                close_PFB=float(base[(base['Date']==day) & (base['Hora_COL']==15) & (base['Minuto']==1)]['PFB_close'])
                if cib_eq-close_PFB>=margen:
                    base['Arbitraje_BL_SE'].iloc[i]=1
                else:
                    base['Arbitraje_BL_SE'].iloc[i]=0
            
            
            
    return base
    

In [127]:
X=Arbitraje(base_all,20)

C:\Users\fract\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [128]:
X

,Date,Hora_COL,Minuto,Hora_NY,Festivo_COL,Festivo_NYSE,Next_Day,Subasta,PFB_ABIERTO,CIB_ABIERTO,...,USD_BID,USD_ASK,ASK_CIB,BID_CIB,ASK_PFB,BID_PFB,Spread_BE_SL,Spread_BL_SE,Arbitraje_BE_SL,Arbitraje_BL_SE
0,2022-01-03,9.0,54.0,9,False,False,0,0,1,1,...,4073.00,4076.90,31.74,31.55,32300.0,32010,-352.3238,-186.30125,0,0
1,2022-01-03,9.0,55.0,9,False,False,0,0,1,1,...,4072.72,4076.00,31.75,31.58,32300.0,32290,-75.37175,-157.96754,0,0
2,2022-01-03,9.0,56.0,9,False,False,0,0,1,1,...,4072.72,4076.00,31.76,31.6,32300.0,32290,-85.563,-137.60644,0,0
3,2022-01-03,9.0,57.0,9,False,False,0,0,1,1,...,4072.72,4075.00,31.76,31.6,32300.0,32290,-77.621,-137.60644,0,0
4,2022-01-03,9.0,58.0,9,False,False,0,0,1,1,...,4072.00,4075.00,31.76,31.62,32300.0,32290,-77.621,-122.9355,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50401,2022-05-12,12.0,55.0,13,False,False,0,0,1,1,...,4114.50,4116.00,36.65,36.56,37690.0,37610,-115.66425,-96.268,0,0
50402,2022-05-12,12.0,56.0,13,False,False,0,0,1,1,...,4115.00,4116.00,36.65,36.56,37680.0,37610,-115.66425,-81.699,0,0
50403,2022-05-12,12.0,57.0,13,False,False,0,0,1,1,...,4112.50,4114.50,36.67,36.6,37680.0,37610,-122.4925,-63.424,0,0
50404,2022-05-12,12.0,58.0,13,False,False,0,0,1,1,...,4112.50,4113.98,36.69,36.59,37680.0,37610,-138.29676,-73.704,0,0


In [129]:
#numero de arbitrajes compra en el exterior/ venta local
X['Arbitraje_BE_SL'].value_counts()

0    47259
1     3147
Name: Arbitraje_BE_SL, dtype: int64

In [130]:
#número de arbitraje compra local/ venta en el exterior
X['Arbitraje_BL_SE'].value_counts()

0    44086
1     6320
Name: Arbitraje_BL_SE, dtype: int64

In [131]:
X.to_excel("arb.xlsx")

In [132]:
retrasos=base_lags(X,3)

In [133]:
#Definimos una función para crear una base de lags que utilizaremos más adelante
def crear_lags(base,lags):
    solo_lags=hist_lags(X,lags)
    solo_lags['Arbitraje_BL_SE']=X['Arbitraje_BL_SE'].iloc[lags:]
    solo_lags['Arbitraje_BE_SL']=X['Arbitraje_BE_SL'].iloc[lags:]
    return solo_lags

In [134]:
solo_lags=crear_lags(X,3)

In [135]:
retrasos.to_excel("retrasos_arb.xlsx")
solo_lags.to_excel("lags.xlsx")

In [136]:
retrasos.shape

(50403, 64)

In [137]:
solo_lags.shape

(50403, 41)

In [138]:
retrasos.head()

,Date,Hora_COL,Minuto,Hora_NY,Festivo_COL,Festivo_NYSE,Next_Day,Subasta,PFB_ABIERTO,CIB_ABIERTO,...,lag_3_ASK_PFB,lag_1_BID_PFB,lag_2_BID_PFB,lag_3_BID_PFB,lag_1_Spread_BE_SL,lag_2_Spread_BE_SL,lag_3_Spread_BE_SL,lag_1_Spread_BL_SE,lag_2_Spread_BL_SE,lag_3_Spread_BL_SE
3,2022-01-03,9.0,57.0,9,False,False,0,0,1,1,...,32300.0,32290,32290,32010,-85.563,-75.37175,-352.3238,-137.60644,-157.96754,-186.30125
4,2022-01-03,9.0,58.0,9,False,False,0,0,1,1,...,32300.0,32290,32290,32290,-77.621,-85.563,-75.37175,-137.60644,-137.60644,-157.96754
5,2022-01-03,9.0,59.0,9,False,False,0,0,1,1,...,32300.0,32290,32290,32290,-77.621,-77.621,-85.563,-122.9355,-137.60644,-137.60644
6,2022-01-03,10.0,0.0,10,False,False,0,0,1,1,...,32300.0,32290,32290,32290,-69.2819,-77.621,-77.621,-122.9355,-122.9355,-137.60644
7,2022-01-03,10.0,1.0,10,False,False,0,0,1,1,...,32300.0,32290,32290,32290,-61.737,-69.2819,-77.621,-94.09,-122.9355,-122.9355


In [139]:
solo_lags.head()

,lag_1_CIB_close,lag_2_CIB_close,lag_3_CIB_close,lag_1_COP_close,lag_2_COP_close,lag_3_COP_close,lag_1_PFB_close,lag_2_PFB_close,lag_3_PFB_close,lag_1_SPX_close,...,lag_2_BID_PFB,lag_3_BID_PFB,lag_1_Spread_BE_SL,lag_2_Spread_BE_SL,lag_3_Spread_BE_SL,lag_1_Spread_BL_SE,lag_2_Spread_BL_SE,lag_3_Spread_BL_SE,Arbitraje_BL_SE,Arbitraje_BE_SL
3,31.6697,31.58,31.58,4075.0,4075.0,4075.0,32300,32000,32000,36366.59,...,32290,32010,-85.563,-75.37175,-352.3238,-137.60644,-157.96754,-186.30125,0,0
4,31.73,31.6697,31.58,4073.86,4075.0,4075.0,32300,32300,32000,36389.77,...,32290,32290,-77.621,-85.563,-75.37175,-137.60644,-137.60644,-157.96754,0,0
5,31.68,31.73,31.6697,4072.72,4073.86,4075.0,32300,32300,32300,36389.36,...,32290,32290,-77.621,-77.621,-85.563,-122.9355,-137.60644,-137.60644,0,0
6,31.68,31.68,31.73,4072.0,4072.72,4073.86,32300,32300,32300,36356.86,...,32290,32290,-69.2819,-77.621,-77.621,-122.9355,-122.9355,-137.60644,0,0
7,31.62,31.68,31.68,4072.0,4072.0,4072.72,32300,32300,32300,36351.53,...,32290,32290,-61.737,-69.2819,-77.621,-94.09,-122.9355,-122.9355,0,0
